In [1]:
!pip install wandb -qqq
!pip install tensorflow-addons

In [2]:
class Config:
    N_SPLITS = 5
    N_EPOCHS = 275
    BATCH_SIZE = 128
    CAT_DIM = 256
    START_LR = 3e-4
    END_LR = 2e-5
    LSTM_SIZE = 512
    DENSE_SIZE = 512
    N_LSTM_LAYERS = 3
    SEQUENCE_LENGTH = 39
    DROPOUT = 0.5
    EMBEDDING_DROPOUT = 0.1
    WANDB_ENABLED = True
    WANDB_API_KEY = '4dad92fd17d935955886d8b751df8aa51f7a8523'
    VER = 'v45'
    SAMPLE_TRAIN_BREATH_IDS = None
    SAMPLE_TEST_BREATH_IDS = None
    DATA_PATH = '/content/drive/MyDrive/kaggle/pressure prediction'
    OUTPUT_PATH = '/content/drive/MyDrive/kaggle/pressure prediction'

In [3]:
# class Config:
#     N_SPLITS = 2
#     N_EPOCHS = 5
#     BATCH_SIZE = 128
#     CAT_DIM = 256
#     START_LR = 3e-4
#     END_LR = 2e-5
#     LSTM_SIZE = 512
#     DENSE_SIZE = 512
#     N_LSTM_LAYERS = 3
#     SEQUENCE_LENGTH = 39
#     DROPOUT = 0.5
#     EMBEDDING_DROPOUT = 0.1
#     WANDB_ENABLED = False
#     WANDB_API_KEY = '4dad92fd17d935955886d8b751df8aa51f7a8523'
#     VER = 'v45'
#     SAMPLE_TRAIN_BREATH_IDS = 100
#     SAMPLE_TEST_BREATH_IDS = 100
#     DATA_PATH = '/content/drive/MyDrive/kaggle/pressure prediction'
#     OUTPUT_PATH = '/content/drive/MyDrive/kaggle/pressure prediction'

In [4]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, StandardScaler
from tqdm import tqdm
from time import time
import os
import gc

import tensorflow as tf
from functools import partial

import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

In [5]:
submission = pd.read_csv(Config.DATA_PATH + os.sep + 'sample_submission.csv')
train = pd.read_csv(Config.DATA_PATH + os.sep + 'train.csv')
test = pd.read_csv(Config.DATA_PATH + os.sep + 'test.csv')

In [6]:
fltr_out = train['u_out'] == 0
target = train['pressure']

In [7]:
train['u_in_2'] = train['u_in'].copy()
test['u_in_2'] = test['u_in'].copy()
fltr = train['u_out'] == 0
train.loc[fltr_out, 'u_in_2'] = 0
test.loc[fltr_out, 'u_in_2'] = 0

In [8]:
oof_df = train[['id']].copy()

In [9]:
SAMPLE_TRAIN = False
SAMPLE_TEST = False

In [10]:
np.random.seed(10)

if Config.SAMPLE_TRAIN_BREATH_IDS is not None:
    sampled = np.random.choice(train['breath_id'].unique(), size=Config.SAMPLE_TRAIN_BREATH_IDS, replace=False)
    print(train.shape)
    train = train[train['breath_id'].isin(sampled)].reset_index(drop=True)
    print(train.shape)

if Config.SAMPLE_TEST_BREATH_IDS is not None:
    sampled = np.random.choice(test['breath_id'].unique(), size=Config.SAMPLE_TEST_BREATH_IDS, replace=False)
    test = test[test['breath_id'].isin(sampled)].reset_index(drop=True)

In [11]:
train['bidc'] = train.groupby('breath_id').cumcount() + 1
test['bidc'] = test.groupby('breath_id').cumcount() + 1

In [12]:
for df_tmp in [train, test]:
    df_tmp['R_cat'] = df_tmp['R'].map({20: 1, 50: 2, 5: 0})
    df_tmp['C_cat'] = df_tmp['C'].map({20: 1, 50: 2, 10: 0})
    df_tmp['RC_cat'] = df_tmp['R'].astype('str')+'-' + df_tmp['C'].astype('str')

mapper = pd.Series(index=train['RC_cat'].unique(), data=np.arange(train['RC_cat'].nunique()))
for df_tmp in [train, test]:
    df_tmp['RC_cat'] = df_tmp['RC_cat'].map(mapper)


cat_cols = ['R_cat', 'C_cat', 'RC_cat']
cat_cols_unq_dct = {}
for c in cat_cols:
    cat_cols_unq_dct[c] = train[c].nunique()

In [13]:
c = 'u_in'
LAG_WINDOW_RANGE = range(3)
train = train.assign(**{f'{c}_t-{t}': train.groupby('breath_id')[c].shift(t) for t in LAG_WINDOW_RANGE})
test = test.assign(**{f'{c}_t-{t}': test.groupby('breath_id')[c].shift(t) for t in LAG_WINDOW_RANGE})
use_fts = [f'{c}_t-{t}' for t in LAG_WINDOW_RANGE]

In [14]:
for df_tmp in [train, test]:
    df_tmp['bidc'] = df_tmp.groupby('breath_id').cumcount()
    df_tmp['u_in_lag_1'] = df_tmp.groupby('breath_id')['u_in'].shift(1).fillna(-999)
    df_tmp['u_in_lag_2'] = df_tmp.groupby('breath_id')['u_in'].shift(2).fillna(-999)
    df_tmp['u_in_lag_3'] = df_tmp.groupby('breath_id')['u_in'].shift(3).fillna(-999)
    df_tmp['u_in_lag_4'] = df_tmp.groupby('breath_id')['u_in'].shift(4).fillna(-999)

    df_tmp['u_in_cumsum'] = df_tmp.groupby('breath_id')['u_in'].cumsum()
    df_tmp['u_in_cumsum_lag_1'] = df_tmp.groupby('breath_id')['u_in_cumsum'].shift(1).fillna(-999)
    df_tmp['u_in_cumsum_lag_2'] = df_tmp.groupby('breath_id')['u_in_cumsum'].shift(2).fillna(-999)
    df_tmp['u_in_cumsum_lag_1-u_in_cumsum_lag_2'] = df_tmp['u_in_cumsum_lag_1'] - df_tmp['u_in_cumsum_lag_2']

    df_tmp['u_in_cumsum_lag_3'] = df_tmp.groupby('breath_id')['u_in_cumsum'].shift(3).fillna(0)
    df_tmp['u_in_cumsum_lag_4'] = df_tmp.groupby('breath_id')['u_in_cumsum'].shift(4).fillna(0)

    df_tmp['u_in_cummean'] = df_tmp['u_in_cumsum']/(df_tmp.groupby('breath_id')['u_in'].cumcount() + 1)
    df_tmp['u_in_cummax'] = df_tmp.groupby('breath_id')['u_in'].cummax()
    df_tmp['next_u_in'] = df_tmp.groupby('breath_id')['u_in'].shift(-1).fillna(0)

    df_tmp['area'] = df_tmp['time_step'] * df_tmp['u_in']
    df_tmp['area'] = df_tmp.groupby('breath_id')['area'].cumsum()
    df_tmp['area_lag_1'] = df_tmp.groupby('breath_id')['area'].shift(1).fillna(0)
    df_tmp['area_lag_2'] = df_tmp.groupby('breath_id')['area'].shift(2).fillna(0)
    df_tmp['area_lead_1'] = df_tmp.groupby('breath_id')['area'].shift(-1).fillna(0)
    df_tmp['area_lead_2'] = df_tmp.groupby('breath_id')['area'].shift(-2).fillna(0)
    df_tmp['area_diff_lag_1'] = df_tmp['area'] - df_tmp['area_lag_1']
    df_tmp['area_diff_lead_1'] = df_tmp['area'] - df_tmp['area_lead_1']


    df_tmp['u_in_cumsum*time_step'] = df_tmp['u_in_cumsum']*df_tmp['time_step']
    df_tmp['u_in_cumsum*time_step_lag_1'] = df_tmp.groupby('breath_id')['u_in_cumsum*time_step'].shift(1).fillna(0)
    df_tmp['u_in_cumsum*time_step/c'] = df_tmp['u_in_cumsum*time_step']/df_tmp['C']
    df_tmp['u_in_cumsum*time_step/c_lag_1'] = df_tmp.groupby('breath_id')['u_in_cumsum*time_step/c'].shift(1).fillna(0)
    df_tmp['area/c'] = df_tmp['area']/df_tmp['C']
    df_tmp['area/c_lag_1'] = df_tmp.groupby('breath_id')['area/c'].shift(1).fillna(0)
    
    df_tmp['u_out_lag_1'] = df_tmp.groupby('breath_id')['u_out'].shift(1).fillna(0)

    df_tmp['time_step*u_out']= df_tmp['time_step']*df_tmp['u_out']

    df_tmp['R+C'] = df_tmp['R'] + df_tmp['C']
    df_tmp['R/C'] = df_tmp['R'] / df_tmp['C']
    df_tmp['u_in/C'] = df_tmp['u_in'] / df_tmp['C']
    df_tmp['u_in/R'] = df_tmp['u_in'] / df_tmp['R']
    df_tmp['u_in_cumsum/C'] = df_tmp['u_in_cumsum'] / df_tmp['C']
    df_tmp['u_in_cumsum/R'] = df_tmp['u_in_cumsum'] / df_tmp['R']
    df_tmp['area*R/C'] = df_tmp['area'] * df_tmp['R/C']
    df_tmp['u_in_cumsum*R/C'] = df_tmp['u_in_cumsum'] * df_tmp['R/C']
    df_tmp['u_in_cumsum*R/C_lag_1'] = df_tmp.groupby('breath_id')['u_in_cumsum*R/C'].shift(1).fillna(0)


    df_tmp['timestep_diff'] = (df_tmp['time_step'] - df_tmp.groupby('breath_id')['time_step'].shift(1)).fillna(0)
    df_tmp['u_in_diff'] = (df_tmp['u_in'] - df_tmp.groupby('breath_id')['u_in'].shift(1)).fillna(0)
    df_tmp['u_in_pct_change'] = (df_tmp['u_in_diff']/(df_tmp['u_in_lag_1'] + 1e-4)).fillna(0)
    df_tmp['u_in_diff_next'] = (df_tmp['u_in'] - df_tmp.groupby('breath_id')['u_in'].shift(-1)).fillna(0)
    df_tmp['u_in_log'] = np.log1p(df_tmp['u_in'])
    df_tmp['u_in_cumsum_log'] = np.log1p(df_tmp['u_in_cumsum'])
    df_tmp['u_in_lag_1_is_zero'] = (df_tmp['u_in_lag_1'] == 0)
    df_tmp['u_in_zero'] = (df_tmp['u_in_lag_1'] == 0)
    df_tmp['u_in_lead_1'] = df_tmp.groupby('breath_id')['u_in'].shift(-1)
    df_tmp['maop'] = df_tmp['bidc'] - df_tmp['breath_id'].map(df_tmp[df_tmp['u_in_zero']].groupby('breath_id')['bidc'].min())
    df_tmp['spike'] = (df_tmp['u_in'] > df_tmp['u_in_lag_1']) & (df_tmp['u_in'] > df_tmp['u_in_lead_1'])
    df_tmp['u_in_lag_1_is_zero_cumsum'] = df_tmp.groupby('breath_id')['u_in_lag_1_is_zero'].cumsum()
    df_tmp['is_max_u_in'] = df_tmp['u_in'] == df_tmp.groupby('breath_id')['u_in'].transform('max')
    df_tmp['nki'] = df_tmp['bidc'] - df_tmp['breath_id'].map(df_tmp.groupby('breath_id')['u_in'].apply(np.argmax))
    df_tmp['nki2'] = df_tmp['bidc'] - df_tmp['breath_id'].map(df_tmp.groupby('breath_id')['u_in_cumsum'].apply(np.argmax))
    df_tmp['nki3'] = df_tmp['nki'] - df_tmp['nki2']
    df_tmp['nki4'] = df_tmp['bidc'] - df_tmp['breath_id'].map(df_tmp[df_tmp['u_in_lag_1_is_zero']].groupby('breath_id')['bidc'].max())
    df_tmp['u_in_cummax - u_in'] = df_tmp['u_in_cummax'] - df_tmp['u_in']
    

In [15]:


train = train[train['bidc'] <= Config.SEQUENCE_LENGTH].reset_index(drop=True)
test = test[test['bidc'] <= Config.SEQUENCE_LENGTH].reset_index(drop=True)

sample_oof_df = train[['id']].copy()
sample_preds_df = test[['id']].copy()

In [16]:
num_cols = [c for c in train.columns if c not in ['id', 'pressure', 'breath_id', 'u_out'] + cat_cols]
train[num_cols] = train[num_cols].fillna(0)
test[num_cols] = test[num_cols].fillna(0)

RS = StandardScaler()
train[num_cols] = RS.fit_transform(train[num_cols])
test[num_cols] = RS.transform(test[num_cols])

In [17]:
N_CLASSES = train['pressure'].nunique()

In [18]:
class VentilatorDataset:
    def __init__(self, df, cat_cols, num_cols, is_train=True):

        if is_train:
            self.inv_mapper = df['pressure'].drop_duplicates().sort_values().reset_index(drop=True).to_dict()
            self.mapper = {val: key for key, val in self.inv_mapper.items()}
            df['pressure_int'] = df['pressure'].map(self.mapper)
            self.pressures_int = df[['pressure_int']].to_numpy().reshape(-1, df['bidc'].nunique())
            self.pressures = df[['pressure']].to_numpy().reshape(-1, df['bidc'].nunique())
            _ = gc.collect()

        self.u_outs = df[['u_out']].to_numpy().reshape(-1, df['bidc'].nunique())
        self.inputs = df[num_cols + cat_cols].values.reshape(-1, df['bidc'].nunique(), len(num_cols + cat_cols)).astype(np.float32)

In [19]:
%%time
test_data = VentilatorDataset(test, cat_cols, num_cols, is_train=False)
train_data = VentilatorDataset(train, cat_cols, num_cols, is_train=True)
_ = gc.collect()

CPU times: user 3.37 s, sys: 9.16 ms, total: 3.38 s
Wall time: 3.35 s


In [20]:
del train
del test
_ = gc.collect()

In [21]:
train_data.inputs.shape, train_data.pressures.shape, train_data.u_outs.shape

((75450, 40, 67), (75450, 40), (75450, 40))

In [22]:
test_data.inputs.shape, test_data.u_outs.shape

((50300, 40, 67), (50300, 40))

In [23]:
NUMERICAL_INPUTS_CNT = len(num_cols)
N_FEATS = train_data.inputs.shape[2]
WINDOW_SIZE=train_data.inputs.shape[1]

NUMERICAL_INPUTS_CNT

64

In [24]:
mapper = train_data.inv_mapper
def map_class_to_cont(x):
    return mapper[x]

In [25]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [26]:
class WeightedSum(tf.keras.layers.Layer):
    """A custom keras layer to learn a weighted sum of tensors"""

    def __init__(self, **kwargs):
        super(WeightedSum, self).__init__(**kwargs)

    def build(self, input_shape=1):
        self.a = self.add_weight(
            name='alpha',
            shape=(),
            initializer='ones',
            dtype='float32',
            trainable=True)

    def call(self, model_outputs):
        return self.a * model_outputs[0] + model_outputs[1]

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [27]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = inputs
    x = tfa.layers.MultiHeadAttention(
        head_size=head_size,
        num_heads=num_heads,
        use_projection_bias = False,
        dropout=Config.DROPOUT
    )([x, x, x])
    res = WeightedSum()([x, inputs])
    res = tf.keras.layers.LayerNormalization(epsilon=1e-6)(res)

    # Feed Forward Part
    x = tf.keras.layers.Dense(ff_dim, activation="relu")(x)
    x = tf.keras.layers.Dropout(Config.DROPOUT)(x)
    x = tf.keras.layers.Dense(inputs.shape[-1])(x)
    x = tf.keras.layers.Dropout(Config.DROPOUT)(x)
    x = WeightedSum()([x, res])
    return tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

In [28]:
def custom_mean_absolute_error(y_true, y_pred, w_out):
    return K.mean(K.abs(y_pred - y_true))

def custom_metric(y_true, y_pred, w_out):
    return mae(y_true[w_out == 0], y_pred[w_out == 0])

def get_LSTM_model(Config):
    w_out = tf.keras.layers.Input(shape=(WINDOW_SIZE, 1))
    targets = tf.keras.layers.Input(shape=(WINDOW_SIZE, 1))
    inputs = tf.keras.layers.Input(shape=(WINDOW_SIZE, N_FEATS))
    
    num_inputs = inputs[:, :, :NUMERICAL_INPUTS_CNT]
    cat_inputs = inputs[:, :, NUMERICAL_INPUTS_CNT:]

    initializer = tf.keras.initializers.glorot_uniform(seed=66)

    all_conv_layers = []
    for sz in [2, 3, 5, 7, 11, 15]:
        conv_curr = tf.keras.layers.Conv1D(16, kernel_size=(sz,), padding='same', use_bias=False)(num_inputs)
        all_conv_layers.append(conv_curr)
    conv_op = tf.keras.layers.Concatenate()(all_conv_layers)
    num_inputs = tf.keras.layers.Concatenate()([num_inputs, conv_op])


    for i, c in enumerate(cat_cols):
      cat_inp = cat_inputs[:, :, i:i+1]
      embed = tf.keras.layers.Embedding(input_dim=cat_cols_unq_dct[c]+2, output_dim=Config.CAT_DIM, embeddings_initializer=initializer)(cat_inp)
      reshaped = tf.reshape(embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3]))
      reshaped = tf.keras.layers.SpatialDropout1D(Config.EMBEDDING_DROPOUT, seed=2)(reshaped)
      num_inputs = tf.keras.layers.Concatenate(axis=2)([reshaped, num_inputs])
    
    
    x = num_inputs
    for i in range(12):
        x = transformer_encoder(x, 8, 128, 1024, 0)
         
    x = tf.keras.layers.Dense(Config.DENSE_SIZE, activation='relu')(x)
    x = tf.keras.layers.Dropout(Config.DROPOUT)(x)
    out = tf.keras.layers.Dense(N_CLASSES)(x)
    
    model = tf.keras.Model(inputs=[inputs, w_out], outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
    
    return model

In [29]:
get_LSTM_model(Config).count_params()

69499982

In [30]:
def get_preds(best_model, X_test, wt_test, STEP=5000):
    test_preds = []
    start_idx = 0

    for start in range(0, X_test.shape[0], STEP):
        preds = best_model.predict([X_test[start: start+STEP], wt_test[start: start+STEP]], verbose=0,
                                   batch_size=512, use_multiprocessing=False)
        test_preds.append(preds)

    test_preds = np.concatenate(test_preds, axis=0)
    return test_preds

In [31]:

class IntervalEvaluation(tf.keras.callbacks.Callback):
    def __init__(self, validation_data=(), interval=10, save_model=True, Config={}, fold=0):

        super(tf.keras.callbacks.Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val, self.w_out_val = validation_data
        self.best_score = np.inf
        self.save_model = save_model
        self.learning_rates = pd.Series(index=np.arange(Config.N_EPOCHS),
                                        data=np.linspace(Config.END_LR,
                                                         Config.START_LR,
                                                         num=Config.N_EPOCHS)[::-1]).to_dict()

    def on_epoch_begin(self, epoch, logs={}):
        self.start_time = time()

        if epoch in self.learning_rates:
            tf.keras.backend.set_value(self.model.optimizer.lr, self.learning_rates[epoch])

        self.curr_lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        
    def on_epoch_end(self, epoch, logs={}):
        keys = list(logs.keys())

        y_pred = get_preds(self.model, self.X_val, self.w_out_val, STEP=3000)
        y_pred = np.vectorize(map_class_to_cont)(np.argmax(y_pred, axis=2))

        val_score = custom_metric(self.y_val.flatten(), y_pred.flatten(), self.w_out_val.flatten())
        
        if val_score < self.best_score:
            self.best_score = val_score
            if self.save_model and epoch > 150: ##### Saving models takes time, so don't save for the first N_EPOCHS // 2
                tf.keras.models.save_model(self.model, f'best_model', save_format="h5")

        total_time = round(time() - self.start_time, 2)
        if Config.WANDB_ENABLED:
            wandb.log({f'Fold {fold} epoch': epoch, f"Fold {fold} train_loss": logs['loss'], f"Fold {fold} val_score": val_score, f'Fold {fold} best_val_score': self.best_score, f'Fold {fold} total_time_in_seconds': total_time})
        
        if epoch % self.interval == 0:
            print(f"Epoch: {epoch:03d} curr_lr: {self.curr_lr:.1e} - train_loss: {logs['loss']:.03f} val_score: {val_score:.03f}  best_val_score: {self.best_score:.03f}  last_epoch t={total_time}s")

In [32]:
Config_dct = {key:value for key, value in Config.__dict__.items() if not key.startswith('__') and not callable(key)}
if Config.WANDB_ENABLED:
    import wandb
    wandb.login(key=Config.WANDB_API_KEY)
    wandb.init(project="google-brain",
               name=f'nikhil_pressure_prediction_dl_{Config.VER}',
               save_code=True,
               allow_val_change=True,
               config=Config_dct)

wandb: Currently logged in as: pressureprediction (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
history_lst = []
oofs = np.zeros(train_data.pressures.shape)
preds_lst = []
kf = KFold(n_splits=Config.N_SPLITS, shuffle=True, random_state=101)
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():

    for fold, (trn_idx, val_idx) in enumerate(kf.split(train_data.pressures, train_data.pressures)):

            print('-'*15, '>', f'Fold {fold}', '<', '-'*15, '\n')
            X_trn, X_val = train_data.inputs[trn_idx], train_data.inputs[val_idx]
            y_trn, y_val = train_data.pressures[trn_idx], train_data.pressures[val_idx]
            y_trn_int, y_val_int = train_data.pressures_int[trn_idx], train_data.pressures_int[val_idx]
     
            wt_trn, wt_val = train_data.u_outs[trn_idx], train_data.u_outs[val_idx]

            model = get_LSTM_model(Config)

            print_val_results = IntervalEvaluation(
                validation_data=(X_val, y_val, wt_val),
                interval=1,
                Config=Config,
                fold=fold,
                )

            history = model.fit([X_trn, wt_trn], y_trn_int,
                        verbose=0,
                        epochs=Config.N_EPOCHS,
                        batch_size=Config.BATCH_SIZE,
                        callbacks=[print_val_results]
                        )

            history_lst.append(history)
            del model, X_trn, y_trn, wt_trn, print_val_results
            _ = gc.collect()

            custom_objects = {
                'custom_mean_absolute_error': custom_mean_absolute_error,
                'WeightedSum': WeightedSum,
            }

            best_model = load_model('best_model', custom_objects=custom_objects)

            vp = get_preds(best_model, X_val, wt_val, STEP=3000)
            vp = np.vectorize(map_class_to_cont)(np.argmax(vp, axis=2))

            oofs[val_idx] = vp

            val_score = round(custom_metric(y_val.flatten(), vp.flatten(), wt_val.flatten()), 4)
            print(f'\nMAE val: {val_score}')

            tp = get_preds(best_model, test_data.inputs, test_data.u_outs, STEP=3000)
            tp = np.vectorize(map_class_to_cont)(np.argmax(tp, axis=2))
            
            preds_lst.append(tp)

            del X_val, y_val, wt_val, best_model, vp, tp
            _ = gc.collect()

    oof_score = round(custom_metric(train_data.pressures.flatten(), oofs.flatten(), train_data.u_outs.flatten()), 4)
    print(f'\nMAE OOF: {oof_score}')

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.20.89.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.20.89.242:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


--------------- > Fold 0 < --------------- 

Epoch: 000 curr_lr: 3.0e-04 - train_loss: 4.628 val_score: 1.052  best_val_score: 1.052  last_epoch t=202.72s
Epoch: 001 curr_lr: 3.0e-04 - train_loss: 3.818 val_score: 0.783  best_val_score: 0.783  last_epoch t=54.47s
Epoch: 002 curr_lr: 3.0e-04 - train_loss: 3.584 val_score: 0.676  best_val_score: 0.676  last_epoch t=55.13s
Epoch: 003 curr_lr: 3.0e-04 - train_loss: 3.447 val_score: 0.594  best_val_score: 0.594  last_epoch t=54.58s
Epoch: 004 curr_lr: 3.0e-04 - train_loss: 3.341 val_score: 0.548  best_val_score: 0.548  last_epoch t=54.85s
Epoch: 005 curr_lr: 2.9e-04 - train_loss: 3.262 val_score: 0.509  best_val_score: 0.509  last_epoch t=54.67s
Epoch: 006 curr_lr: 2.9e-04 - train_loss: 3.194 val_score: 0.467  best_val_score: 0.467  last_epoch t=54.41s
Epoch: 007 curr_lr: 2.9e-04 - train_loss: 3.137 val_score: 0.433  best_val_score: 0.433  last_epoch t=54.4s
Epoch: 008 curr_lr: 2.9e-04 - train_loss: 3.076 val_score: 0.422  best_val_score: 0

In [ ]:
if Config.WANDB_ENABLED:
    wandb.config.OOF_SCORE = oof_score
    wandb.finish()

In [ ]:
preds = np.median(np.array(preds_lst), axis=0)
sample_oof_df['pressure'] = oofs.flatten()
sample_preds_df['pressure'] = preds.flatten()

In [ ]:
submission = pd.merge(submission[['id']], sample_preds_df, on='id', how='left')
submission['pressure'] = submission['pressure'].fillna(0)

In [ ]:
print(oof_df.shape)
oof_df = pd.merge(oof_df[['id']], sample_oof_df, on='id', how='left')
oof_df['pressure'] = oof_df['pressure'].fillna(0)
print(oof_df.shape)

In [ ]:
path = Config.OUTPUT_PATH + os.sep + f'dl_oof_{Config.VER}.csv'
oof_df.to_csv(path, index=False)

In [ ]:
path = Config.OUTPUT_PATH + os.sep + f'dl_sub_{Config.VER}.csv'
submission.to_csv(path, index=False)